## Neural network

### Import the Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

### Create the Neural Network model

In [2]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
#checking for input and output consistency
model = NN(784,10)
x = torch.randn(128,784)
print(model(x).shape)

torch.Size([128, 10])


### Set the device

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### Intialize the hyperparameters

In [5]:
input_size = 784
num_classes = 10
batch_size = 64
learning_rate = 0.001
num_epochs = 1 

### Load Data

In [6]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)

In [7]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [8]:
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

### Initialize Network

In [9]:
model = NN(input_size=input_size, num_classes = num_classes).to(device)

### Loss and Optimizer

In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train Network

In [20]:

for epoch in range(num_epochs):
    print(f'Epoch:{epoch}')
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        data = data.reshape(data.shape[0], -1)
        
        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
            
    
        

Epoch:0


### Check accuracy on training 

In [23]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            
            x= x.reshape(x.shape[0], -1)
            
            scores = model(x)
            _, predictions = scores.max(1)
            
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        model.train()
        #return acc

In [24]:
#check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 9627/10000 with accuracy 96.27
